In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from spark.preprocessor import preprocessor
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model, regularizers, layers
import tensorflow.keras as keras
from tensorflow.keras.callbacks import EarlyStopping, 
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score


In [2]:
X_train, X_test, y_train, y_test = preprocessor("~/code/marzecovaa/spark/processed_data/merged_dfq_v2.csv")

In [ ]:
def initialize_model(input_shape:tuple) -> Model:
    """
    Initialize the Neural Network with random weights
    """

    reg = regularizers.l1_l2(l1=0.005)

    model = Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(100, activation="relu", kernel_regularizer=reg))
    model.add(layers.BatchNormalization(momentum=0.9))
    model.add(layers.Dropout(rate=0.1))
    model.add(layers.Dense(50, activation="relu"))
    model.add(layers.BatchNormalization(momentum=0.9))
    model.add(layers.Dropout(rate=0.1))
    model.add(layers.Dense(3, activation="softmax"))

    return model

In [4]:
model = initialize_model(input_shape=X_train.shape[1:])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │         3,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 50)             │           200 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           153 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,703 (37.90 KB)

 Trainable params: 9,403 (36.73 KB)

 Non-trainable params: 300 (1.17 KB)

In [5]:
batch_size = 4


model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
es = EarlyStopping(
    monitor="val_loss",
    patience=20,
    restore_best_weights=True,
    verbose=0
)



history = model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    epochs=500,
    batch_size=batch_size,
    callbacks=[es, reduce_lr],
    verbose=1
)


Epoch 1/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4433 - loss: 3.0803 - val_accuracy: 0.5600 - val_loss: 2.9656
Epoch 2/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - accuracy: 0.5600 - loss: 2.8015 - val_accuracy: 0.5733 - val_loss: 2.9065
Epoch 3/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 688us/step - accuracy: 0.6300 - loss: 2.6121 - val_accuracy: 0.6267 - val_loss: 2.6432
Epoch 4/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.6767 - loss: 2.5077 - val_accuracy: 0.6267 - val_loss: 2.6076
Epoch 5/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step - accuracy: 0.7033 - loss: 2.3926 - val_accuracy: 0.6667 - val_loss: 2.4862
Epoch 6/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.6967 - loss: 2.3879 - val_accuracy: 0.7733 - val_loss: 2.3314
Epoch 7/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - accuracy: 0.7033 - loss: 2.3215 - val_accuracy: 0.7333 - val_loss: 2.3630
Epoch 8/500
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - accuracy: 0.7400 - loss: 2.2129 - val_accura

In [7]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)


# Metrics
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Balanced Accuracy: 0.6286396574440053
